In [1]:
# necessary packages #

#using Pkg
#Pkg.add("Distances")
using Distributions
using Random
using Distances
using LinearAlgebra
using SparseArrays
using IterativeSolvers
using ProgressMeter
using JLD2
using BenchmarkTools
using StatsPlots

In [2]:
include("../../utils/util2.j")

generate_comparison_table (generic function with 2 methods)

In [3]:
# Set the parameters for sim #

Random.seed!(123);  
N = 2000 # sample size
q = 10; L = 2 

sqrt_Σ_diag = sqrt.([0.5, 1, 0.4, 2, 0.3, 2.5, 3.5, 0.45, 1.5, 0.5]);
β = [1.0 -1.0 1.0 -0.5 2.0 -1.5 0.5 0.3 -2.0 1.5
     -3.0 2.0 2.0 -1.0 -4.0 3.0 4.0 -2.5 5.0 -3.0];
ϕ = [6.0, 9.0]; #[6.0, 9.0] #sim2:[9.0, 6.0]
Λ = rand(Uniform(-1.0, 1.0), L, q);
coords = rand(N, 2);                                          # random location over unit square
X = [fill(1, (N,)) rand(N)];                             # design matrix
Dist = pairwise(Euclidean(), coords, dims = 1);                  # distance matrix
F = [rand(MvNormal(exp.(-ϕ[1] * Dist)), 1) rand(MvNormal(
            exp.(-ϕ[2] * Dist)), 1)];

In [4]:
F0 = F - fill(1.0, N) * mean(F, dims = 1);
std_F = std(F0, dims=1);
F0[:, 1] = F0[:, 1] ./ std_F[1];
F0[:, 2] = F0[:, 2] ./ std_F[2];
cov(F0)

2×2 Matrix{Float64}:
 1.0        0.0834477
 0.0834477  1.0

In [5]:
ω = F0 * Λ;
Y = X * β + ω + [rand(Normal(0.0, sqrt_Σ_diag[i]), 1)[1] for j in 1:N, i in 1:q];
ω_incp = ω + fill(1.0, (N, 1)) * transpose(β[1, :]);
dimY = size(Y); dimX = size(X);
n = dimY[1];
q = dimY[2];
p = dimX[2];
[n q p]

1×3 Matrix{Int64}:
 2000  10  2

In [6]:
round.(std(ω, dims=1), digits = 2)

1×10 Matrix{Float64}:
 0.81  0.49  0.62  0.74  1.14  0.79  1.01  1.31  0.4  1.0

In [7]:
round.(Λ, digits = 2)

2×10 Matrix{Float64}:
  0.81  0.49  -0.49  -0.15  -0.8    0.38  -0.94  0.86   0.16  -0.76
 -0.11  0.02  -0.33   0.74  -0.75  -0.73  -0.3   0.92  -0.38  -0.59

In [8]:
isdir("../data") ## i added this and the line below, since @save wasn't working

true

In [9]:
mkdir("../data")

LoadError: IOError: mkdir("../data"; mode=0o777): file already exists (EEXIST)

In [10]:
@save "../data/sim1.jld" Y X coords n q p F0 β Λ sqrt_Σ_diag;

In [11]:
isdir("../results")
isdir("../results/PCMC") # i added this and hte line below

false

In [12]:
mkdir("../results")
mkdir("../results/PMCMC")

"../results/PMCMC"

In [13]:
using Plots

lon = coords[:, 1]
lat = coords[:, 2]
values = F[:, 1]  # Assuming you want the first column

scatter(lon, lat, zcolor=values, color=:RdBu, markerstrokewidth=0, legend=false,
        markersize=10.0, xlabel="Longitude", ylabel="Latitude", title="Spatial Data Visualization",
        yflip=true);
Plots.savefig("../results/PMCMC/F1.png")

"/Users/tati/Documents/Github/PBSF_lit/sim/results/PMCMC/F1.png"

In [14]:
values = F[:, 2]  # Assuming you want the first column

scatter(lon, lat, zcolor=values, color=:RdBu, markerstrokewidth=0, legend=false,
        markersize=10.0, xlabel="Longitude", ylabel="Latitude", title="Spatial Data Visualization",
        yflip=true);
Plots.savefig("../results/PMCMC/F2.png")

"/Users/tati/Documents/Github/PBSF_lit/sim/results/PMCMC/F2.png"